In [1]:
import sys
sys.path.append("../../src/")
import numpy as np
import math
import uncertainpy.gradual as grad

In [2]:
# set semantics and compute strength of arguments
bag = grad.BAG("../../bags/fig3.bag")

# DF-QuAD semantics
agg_f = grad.semantics.modular.ProductAggregation()
inf_f = grad.semantics.modular.LinearInfluence(conservativeness=1)

# QE semantics
# agg_f = grad.semantics.modular.SumAggregation()
# inf_f = grad.semantics.modular.QuadraticMaximumInfluence(conservativeness=1)

strength_values = grad.algorithms.computeStrengthValues(bag, agg_f, inf_f)

In [3]:
# get all the attacks and supports in QBAF
att_relation = bag.get_attacks()
sup_relation = bag.get_supports()
all_relation = att_relation + sup_relation

In [4]:
# rename relation concisely
all_relation_new = []
for i in all_relation:
    if str(i)[0:3]=="Att":
        all_relation_new.append(("att",str(i.get_attacker().get_name()),str(i.get_attacked().get_name())))
    elif str(i)[0:3]=="Sup":
        all_relation_new.append(("sup",str(i.get_supporter().get_name()),str(i.get_supported().get_name())))
all_relation_new

[('att', 'delta', 'beta'),
 ('att', 'delta', 'gamma'),
 ('att', 'zeta', 'delta'),
 ('sup', 'beta', 'alpha'),
 ('sup', 'gamma', 'alpha')]

In [5]:
# set topic argument
topic_a = "alpha"
topic_arg = bag.arguments[topic_a]

In [6]:
# compute strength for all arguments
for arg in bag.arguments.values():
    print((arg.name,arg.initial_weight,arg.strength))

('alpha', 0.5, 0.8046875)
('beta', 0.5, 0.375)
('gamma', 0.5, 0.375)
('delta', 0.5, 0.25)
('zeta', 0.5, 0.5)


In [7]:
n = len(all_relation_new) # the number of all relations in the AF
attribution = {} # store the attribution of all the relations

for i in range(len(all_relation_new)):

    cur_relation = all_relation_new[i]


    # compute strength before removing
    strength_values = grad.algorithms.computeStrengthValues(bag, agg_f, inf_f)
    topic_arg = bag.arguments[topic_a]
    strength_before_drop_cur = topic_arg.strength


    # remove cur_relation
    if cur_relation[0]=="att":
        temp_attacker = bag.remove_attack(cur_relation[1],cur_relation[2]) # cur_relation[1] attacks cur_relation[2]
    elif cur_relation[0]=="sup":
        temp_supporter = bag.remove_support(cur_relation[1],cur_relation[2]) # cur_relation[1] supports cur_relation[2]


    # compute strength after removing
    strength_values = grad.algorithms.computeStrengthValues(bag, agg_f, inf_f)
    topic_arg = bag.arguments[topic_a]
    strength_after_drop_cur = topic_arg.strength


    # compute strength difference
    difference = strength_before_drop_cur - strength_after_drop_cur


    # compute attributions
    attribution[cur_relation] = difference

    # recover QBAF for the next iteration
    bag = grad.BAG("../../bags/fig3.bag")
    strength_values = grad.algorithms.computeStrengthValues(bag, agg_f, inf_f)

In [8]:
attribution

{('att', 'delta', 'beta'): -0.0390625,
 ('att', 'delta', 'gamma'): -0.0390625,
 ('att', 'zeta', 'delta'): 0.0859375,
 ('sup', 'beta', 'alpha'): 0.1171875,
 ('sup', 'gamma', 'alpha'): 0.1171875}